<a href="https://colab.research.google.com/github/WB-Jang/ML-notebooks/blob/main/Default_prediction/Loan_Default_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 은행 여신 계좌의 Default를 예측하기 위한 Deep Learning Development
(1) Data Set
- FY 2024 all seg loan acct
- Each month-end snapshot
- All columns are divided into 2 group : Numerical, Categorical
-
- Only from 68 columns  

(2) Data Cleansing & Feature Engineering

(3) Data Augmentation for Contrastive Learning

## Pre-training
- Masked Auto-Encoder ver
- Contrastive Learning ver

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuarcy_score, roc_auc_score, f1_score
import numpy as np
import torch.optim as optim
import torch.optim.lr_scheduler import ReduceLROnPlateau

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Encoder(nn.Module):
  # Transformer Encdoer
  def __init__(self, cnt_cat_features, cnt_num_features, cat_max_dict, d_model=32, nhead=4, num_layers=6, dim_feedforward=64, dropout_rate=0.3):
    super().__init__()
    self.d_model = d_model
    self.cnt_cat_features = cnt_cat_features
    self.cnt_num_features = cnt_num_features
  # 범주형 변수 임베딩
    """
    nn.Embedding은 하나의 Lookup table과 같아서, 첫 번째 인자만큼의 행에 두 번째 인자 길이만큼의 차원을 가진 벡터를 각각 생성함.
    예를 들어, nn.Embedding(78,32)라면, 0~77에 해당하는 32차원의 고유한 벡터를 생성하고, 0이 들어오면 첫 번째 행의 32차원 벡터를 출력하고, 7이 들어오면 7번째 행의 32차원 벡터를 출력
    """
    self.embeddings = nn.ModuleList([
        nn.Embedding(cat_max_dict[i],d_model) for i in range(cnt_cat_features)

    ])
  # 수치형 변수 임베딩 : 수치는 실수 크기의 정보를 담은 vector로 변환해야 하므로 nn.Linear를 사용해야 한다
    self.num_embeddings = nn.ModuleList([
        nn.Linear(1,d_model) for _ in range(cnt_num_features)
    ])

  # Transformer 인코더 레이어 정의
    encoder_layer = nn.TransformerEncoderLayer( # TransformerEncoderLayer에는 self-attention이 기본으로 포함. CrossAttention이 필요하다면, nn.TransformerDecoderLayer 사용
        d_model=d_model
        , nhead=nhead
        , dim_feedforward=dim_feedforward
        , dropout=dropout_rate
        , batch_first=True # batch_first 옵션은 입력 차원을 (Sequential_length, batch_size, embedding_dim) -> (batch_size, Sequential_length(컬럼 수), embedding_dim)로 변경. 이렇게 변경하면 각 data point 별로 하나의 tabular data 형식을 가짐
        , norm_first=True # norm_first 옵션은 layer_norm -> residual_connection 순서로 진행한다는 의미. 직관적으로 생각했을 때에는, Residual Connection -> Layer norm을 하게 되면, Residual의 의미가 반감되는 느낌.
    )
    self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

  def forward(self, x_cat, x_num):
    """
    categorical train set은 (미니배치 16, 컬럼 50)의 형태이고, 각 컬럼 별로 뽑아내고 (16,) unsqueeze하면 (16,1) 형식 50개가 된다
    각 컬럼 별로 n개의 distinct value가 있다고 할 때에, nn.Embedding 함수는 (n,32) 형식의 Lookup table을 만들고, 들어오는 입력에 따라 k번째에 해당하는 32차원 벡터를 출력으로 내놓게 된다.
    최종적으로는 (16,1,32) 형태의 3차원 벡터가 출력된다. 이를 torch.cat(dim=1)하게 되면, (16,50,32)가 각 미니배치별로 생성된다.

    Numerical train set을 임베딩할 때에 .unsqueeze가 2번 사용되고 있지만, 첫 번째 unsqueeze는 (16,) -> (16,1)로 변환시키고, 이 벡터가 nn.Linear 함수를 통과하면서 1은 사라지고, (16,32)의 결과물만 남게되고,
    두 번째 .unsqueeze를 통해 (16,1,32) 형태가 되고, torch.cat을 통해 (16,45,32)가 되고, cat_emb와 concat이 가능하게 된다.
    """
    cat_emb = torch.cat([self.embeddings[i](x_cat[:,i]).unsqueeze(1) for i in range(self.cnt_cat_features)], dim=1)
    num_emb = torch.cat([self.num_embeddings[i](x_num[:,i].unsqueeze(1)).unsqueeze(1) for i in range(self.cnt_num_features)], dim=1)
    x = torch.cat([cat_emb,num_emb],dim=1)

    x = self.transformer(x)
    return x

class TabTransformerClassifier(nn.Module):
  """ 사전 학습된 인코더에 분류 어댑터 추가 """
  def __init__(self, encoder, d_model=32, final_hidden=128, dropout_rate=0.3):
    super().__init__()
    self.encoder = encoder
    self.fc = nn.Sequential(
        nn.Linear(d_model, final_hidden)
        , nn.ReLU()
        , nn.Dropout(dropout_rate)
        , nn.Linear(final_hidden, 2)
    )
  def forward(self, x_cat, x_num):
    encoded = self.encoder(x_cat, x_num)
    pooled = encoded.mean(dim=1)
    """
    Transformer encoder의 출력은 (batch_size, seq_len(컬럼 수), embedding_dim)인데, fc에는 (batch_size, embedding_dim)이 들어가야 하므로, 모든 변수(컬럼 혹은 feature)의 출력을 평균내어 대표 벡터 생성
    즉, pooled는 각 샘플(혹은 data point)별 embedding_dim 차원의 대표 벡터를 의미함
    """
    return self.fc(pooled)

class ProjectionHead(nn.Module):
  """ Contrastive Learning을 위한 프로젝션 헤드 """
  def __init__(self, d_model=32, projection_dim=128):
    super().__init__()
    self.head = nn.Sequential(
        nn.Linear(d_model, d_model)
        , nn.ReLU()
        , nn.Linear(d_model, projection_dim)
    )
  def forward(self, x):
    return self.head(x)

def tabular_augment(x_cat, x_num, mask_ratio=0.15):
  """
  Tabulaer data augmentaion 함수
  입력 feature의 일부를 무작위로 마스킹(0으로 만듬)하여 증강된 뷰를 생성함
  """
  B, L_cat = x_cat.shape
  B, L_num = x_num.shape

  # 범주형 증강
  cat_mask = torch.rand(x_cat.shape, device=x_cat.device) < mask_ratio
  x_cat_aug = x_cat.clone()
  x_cat_aug[cat_mask] = 0

  # 수치형 증강
  num_mask = torch.rand(x_num.shape, device=x_num.device) < mask_ratio
  x_num_aug = x_num.clone()
  x_num_aug[num_mask] = 0.0

  return x_cat_aug, x_num_aug

class NTXentLoss(nn.Module):
  """ Contrastive Learning을 위한 NTXent Loss 함수 """
  def __init__(self, temperature=0.5):
    super().__init__()
    self.temperature = temperature
    self.criterion = nn.CrossEntropyLoss()
    self.similarity_f = nn.CosineSimilarity(dim=2)

  def forward(self, z_i, z_j):
    B, _ = z_i.shape
    z = torch.cat([z_i,z_j],dim=0) # z_i와 z_j를 concat하여 (2B,D) 텐서 생성
    sim = self.similarity_f(z.unsqueeze(1),z.unsqueeze(0)) / self.temperature

    sim_i_j = torch.diag(sim, B) # 자기 자신과의 유사도(대각선)를 제외하기 위한 마스킹
    sim_j_i = torch.diag(sim, -B)

    # 긍정 쌍 (positive pairs)
    positive_samples = torch.cat([sim_i_j, sim_j_i], dim=0).reshape(2 * B, 1)

    # 자기 자신과의 비교를 제외한 부정 쌍 (negative pairs)
    mask = torch.ones((2 * B, 2 * B), dtype=bool, device=z.device)
    mask.fill_diagonal_(False)

    negative_samples = sim[mask].reshape(2 * B, -1)

    # 최종 로짓: [positive_sample, negative_samples]
    logits = torch.cat([positive_samples, negative_samples], dim=1)

    # 정답 레이블: 항상 0번째가 긍정 쌍이므로 0으로 채움
    labels = torch.zeros(2 * B, dtype=torch.long, device=z.device)

    return self.criterion(logits, labels)

def pretrain_contrastive_v2(encoder, projection_head, loader, epochs=10, device='cpu'):
    """V2: 대조 학습 사전학습 루프"""
    print("--- Starting V2: Contrastive Learning Pre-training ---")
    encoder.to(device).train()
    projection_head.to(device).train()

    optimizer = torch.optim.Adam(
        list(encoder.parameters()) + list(projection_head.parameters()), lr=3e-4
    )
    criterion = NTXentLoss()

    for epoch in range(epochs):
        total_loss = 0.0
        for batch in loader:
            x_cat, x_num = batch["str"].to(device), batch["num"].to(device)

            # 1. 동일 데이터에 대해 두 개의 다른 증강된 뷰 생성
            x_cat_i, x_num_i = tabular_augment(x_cat, x_num)
            x_cat_j, x_num_j = tabular_augment(x_cat, x_num)

            # 2. 각 뷰를 인코더와 프로젝션 헤드에 통과
            h_i = encoder(x_cat_i, x_num_i).mean(dim=1) # (B, d_model)
            h_j = encoder(x_cat_j, x_num_j).mean(dim=1) # (B, d_model)

            z_i = projection_head(h_i) # (B, projection_dim)
            z_j = projection_head(h_j) # (B, projection_dim)

            # 3. NT-Xent 손실 계산
            loss = criterion(z_i, z_j)

            # 4. 역전파
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"[Pretrain V2] Epoch {epoch+1}, Loss: {total_loss / len(loader):.4f}")



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler